# Flux Intensity

Agustín Vallejo Villegas

July 2021

In this notebook we calculate the flux intensity of Apophis measured by an observatory using bistatic radar estimations. Angular size and speed for Apophis is also calculated. All this in order to better grasp where the ideal site of observation would be for this event.

### Libraries

In [ ]:
"BASIC LIBRARIES"
import numpy as np
import pylab as plt
import pandas as pd

In [ ]:
"ASTROPY"
!pip install astroquery
from astroquery.jplhorizons import Horizons

from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun
from astropy.constants import c
import astropy.units as u

     |████████████████████████████████| 4.4MB 4.3MB/s 
     |████████████████████████████████| 808kB 34.0MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 3.2MB 32.5MB/s 
  Created wheel for mimeparse: filename=mimeparse-0.1.3-cp37-none-any.whl size=3864 sha256=8acc7170d1d735da20fc1471403b41e7a06ee387ebfb1626029832f189ed2081
  Stored in directory: /root/.cache/pip/wheels/54/ca/c7/3db47cc5c748286db22a7fab43ccf985903d2b9ca119de16ab
Successfully built mimeparse


In [ ]:
"TQDM"
!pip install tqdm
from tqdm import tqdm

## Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
!ls "/gdrive/MyDrive/Apophis/Data"

AllRadiotelescopes.xlsx  ephemeris.xlsx  functions.py


# Load Data

In [ ]:
radio =  pd.read_excel("/gdrive/MyDrive/Apophis/Data/AllRadiotelescopes.xlsx",sheet_name='AA')
print(radio)

In [ ]:
print(radio.columns)

In [ ]:
ephemeris = pd.read_excel("/gdrive/MyDrive/Apophis/Data/ephemeris.xlsx")

## Apophis Geographical Coordinates

In [ ]:
def get_lon(TIME,RA):
  """
  This function outputs the earth longitude at which an object with a certain RA
  at a certain jd will be at the meridian, or directly overhead.
  """
  T = Time(TIME,format='jd')
  G = T.sidereal_time('apparent', 'greenwich').to(u.deg)
  return RA - G.value

maxlons = get_lon(ephemeris['datetime_jd'],ephemeris['RA'])
maxlats = ephemeris['DEC']
deltas = ephemeris['delta (Rt)']

for i in range(len(maxlons)):
  if maxlons[i] < -180: maxlons[i] += 360

ephemeris['Longitude'] = maxlons
ephemeris['Latitude'] = maxlats

for i in range(1,len(maxlons)):
  if maxlons[i] > maxlons[i-1]:
    j = i-1


maxlons[j] = np.nan
maxlons = np.insert(maxlons,j,[-180])
maxlats = np.insert(maxlats,j,[maxlats[j]])
deltas = np.insert(deltas,j,[deltas[j]])

pandasephem = ephemeris.to_pandas()
pandasephem.to_excel("/gdrive/MyDrive/Apophis/Data/ephemeris.xlsx")

/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "utcut1" yielded 133 of "dubious year (Note 3)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "utctai" yielded 133 of "dubious year (Note 3)"
  ErfaWarning)


In [ ]:
print("Closest Approach")
ephemeris.to_pandas().loc[np.where(ephemeris['delta'] == min(ephemeris['delta']))][['datetime_str','delta (Rt)','Longitude','Latitude']]

Closest Approach


,datetime_str,delta (Rt),Longitude,Latitude
8,2029-Apr-13 21:40,5.968741,-38.938478,27.2881


## Light Distance

In [ ]:
# Obtains the Astropy Position Vector of given earth coordinates
pos = lambda x,h: EarthLocation(lon=x['Longitude']*u.deg,lat=x['Latitude']*u.deg,height=h*u.earthRad)

# Transforms an Astropy Vector into a 3-tuple
vec = lambda a:  (a.x.value,
                  a.y.value,
                  a.z.value)

# Obtains the difference between two Astropy Vectors
vecdiff = lambda a,b: (a.x.value-b.x.value,
                       a.y.value-b.y.value,
                       a.z.value-b.z.value)

# Obtains the magnitude of a vector
dist = lambda v: np.linalg.norm(v)

# Calculates the angle between two vectors in degrees
angle = lambda v1,v2: np.rad2deg( np.arccos( np.dot(v1,v2) / (dist(v1) * dist(v2)) ) )

def resol_angle(obs):
    if "-" in obs['Frequency Range']:
        print("Antenna Frequency Range:",obs['Frequency Range'])
        freq = float(input("At what frequency will the antenna operate in MHz? "))
    else:
        freq = float(obs['Frequency Range'].split(" ")[0])
    freq *= u.MHz
    lamb = c/(freq)
    diameter = float(obs['Diameter'][:-1]) * u.m
    angle = np.arcsin(1.22 * lamb / diameter)
    return angle

def size_angle(obs,apo):
    APOPHIS_SIZE = 450 * u.m
    pos1 = pos(obs,0)
    posA = pos(apo,apo['delta (Rt)'])
    vec1 = vecdiff(posA,pos1)
    distance = dist(vec1)*u.earthRad
    angle = np.arctan(APOPHIS_SIZE / distance)
    return angle

def reflected_power(obs1,apo,print_info=False):
    if obs1['Power'] is not np.nan:      
        power = float(obs1['Power'][:-2]) * u.kW # Power of emitting antenna
        apparent_angle = size_angle(obs1,apo)    # Apparent angle of Apophis
        resolution_angle = resol_angle(obs1)     # Resolution of emitter
        power_at_apophis = power * (apparent_angle / resolution_angle)**2 # Power reflected by Apophis
        if print_info:
            print("\nEMITTING:",obs1['Name'],obs1['Power'])
            print("RECEIVING:",obs2['Name'])
            print("\nPower reflected by Apophis: {:.2e}".format(power_at_apophis))
        return power_at_apophis
    else:
        print("First Antenna does not emmit!")
        return None

def recieved_power(obs2,apo,reflected,print_info=False):
    APOPHIS_SIZE = 450 * u.m
    pos2 = pos(obs2,0)
    posA = pos(apo,apo['delta (Rt)'])
    vec2 = vecdiff(posA,pos2)
    distance = dist(vec2)*u.earthRad.to(u.m) * u.m
    recieved_flux = reflected / (2*np.pi* distance**2)
    diameter = float(obs2['Diameter'][:-1]) * u.m
    # recieved_power = recieved_flux * (np.pi * (diameter/2)**2) * np.cos(angle(vec(pos2),vec(posA)))
    recieved_power = recieved_flux * abs(np.cos(angle(vec(pos2),vec(posA)))) / (50 * u.MHz)
    recieved_power = recieved_power.to(u.Jansky)
    if print_info:
        print("Power recieved by {:s}: {:.2e}".format(obs2['Name'],recieved_power))
    return recieved_power

In [ ]:
obs1 = radio.iloc[0] # 0 Arecibo
obs2 = radio.iloc[1] # 1 Goldstone
apo = ephemeris[8]
reflected = reflected_power(obs1,apo,print_info=True)
recieved_power(obs2,apo,reflected,print_info=True)

Antenna Frequency Range: 8 MHz - 3,000 MHz.
At what frequency will the antenna operate in MHz? 3000

EMITTING: Arecibo Telescope 2000kW
RECEIVING: Goldstone Radio Telescope

Power reflected by Apophis: 1.68e+00 kW
Power recieved by Goldstone Radio Telescope: 2.56e+05 Jy


<Quantity 255709.04192283 Jy>

In [ ]:
obs1 = radio.iloc[1] # 1 Goldstone
obs2 = radio.iloc[3] # 3 Green Bank
apo = ephemeris[8]
reflected = reflected_power(obs1,apo,print_info=True)
recieved_power(obs2,apo,reflected,print_info=True)


EMITTING: Goldstone Radio Telescope 500kW
RECEIVING: Green Bank Telescope (GBT)

Power reflected by Apophis: 1.53e-01 kW
Power recieved by Green Bank Telescope (GBT): 1.06e+04 Jy


<Quantity 10584.09204438 Jy>

In [ ]:
obs1 = radio.iloc[8] # 8 Camberra
obs2 = radio.iloc[7] # 7 Fast
apo = ephemeris[23] # 23 Máximo en Camberra, 30 Máximo en FAST
reflected = reflected_power(obs1,apo,print_info=True)
recieved_power(obs2,apo,reflected,print_info=True)


EMITTING: Canberra Deep Space Communication Complex (CDSCC) 400kW
RECEIVING: Five hundred meter Aperture Spherical Telescope (FAST)

Power reflected by Apophis: 8.57e-04 kW
Power recieved by Five hundred meter Aperture Spherical Telescope (FAST): 1.16e+01 Jy


<Quantity 11.62723097 Jy>

In [ ]:
obs1 = radio.iloc[12] # 12 GREAT
obs2 = radio.iloc[7] # 7 Fast
apo = ephemeris[30] # 30 Máximo en FAST
reflected = reflected_power(obs1,apo,print_info=True)
recieved_power(obs2,apo,reflected,print_info=True)

Antenna Frequency Range: 7145 MHz - 7235 MHz
At what frequency will the antenna operate in MHz? 7200

EMITTING: GREAT:GRound station for deep space Exploration And Telecommunication 20 kW
RECEIVING: Five hundred meter Aperture Spherical Telescope (FAST)

Power reflected by Apophis: 1.55e-04 kW
Power recieved by Five hundred meter Aperture Spherical Telescope (FAST): 4.49e-02 Jy


<Quantity 0.0449456 Jy>

In [ ]:
obs1 = radio.iloc[9] # 9 Eastern Deep Space Communications Center
obs2 = radio.iloc[7] # 7 Fast
apo = ephemeris[30] # 30 Máximo en FAST
reflected = reflected_power(obs1,apo,print_info=True)
recieved_power(obs2,apo,reflected,print_info=True)


EMITTING: Eastern Deep Space Communications Center 80kW
RECEIVING: Five hundred meter Aperture Spherical Telescope (FAST)

Power reflected by Apophis: 1.53e-06 kW
Power recieved by Five hundred meter Aperture Spherical Telescope (FAST): 4.41e-04 Jy


<Quantity 0.00044119 Jy>